# OpenAI Function Calling 101
One of the struggles of using LLMs like ChatGPT is that they do not produce a structured data output. This is important for programmatic systems that largely rely on structured data for system interaction. For example, if you want to build a program that analyzes the sentiment of a movie review, you might have to execute a prompt that looks like the following:

```
prompt = f'''
Please perform a sentiment analysis on the following movie review:
{MOVIE_REVIEW_TEXT}
Please output your response as a single word: either "Positive" or "Negative". Do not add any extra characters.
'''
```

The problem with this is that it doesn't always work. It's pretty common that the LLM will throw in an undesired period or longer explanation like: "The sentiment of this movie is: Positive." While you can regex out the answer (🤢), this is obviously not ideal. What would be ideal is if the LLM would return the output as something like the following structured JSON:

```
{
    'sentiment': 'positive'
}
```

Enter **OpenAI's new function calling**! Function calling is precisely the answer to the problem above. This Jupyter notebook will demonstrate a simple example of how to use OpenAI's new function calling in Python. If you would like to see the full documentation, [please check out this link](https://platform.openai.com/docs/guides/gpt/function-calling).

## Notebook Setup
Let's start with our imports. Now, you may already have the `openai` Python client already installed, but you'll most likely need to upgrade it to get the new function calling functionality. Here's how to do this upgrade in your Terminal / Powershell with `pip`:

```
pip install openai --upgrade
```

In [1]:
# Importing the necessary Python libraries
import os
import json
import yaml
import openai

In [2]:
# Loading the API key and organization ID from file (NOT pushed to GitHub)
with open('../keys/openai-keys.yaml') as f:
    keys_yaml = yaml.safe_load(f)

# Applying our API key and organization ID to OpenAI
openai.organization = keys_yaml['ORG_ID']
openai.api_key = keys_yaml['API_KEY']
os.environ['OPENAI_API_KEY'] = keys_yaml['API_KEY']

To test out the function calling functionality, I wrote a short "About Me" containing particular facts that we'll be parsing out into appropriate data structures, including integers and strings. Let's load in this text

In [3]:
# Loading the "About Me" text from local file
with open('../data/about-me.txt', 'r') as f:
    about_me = f.read()

print(about_me)

Hello! My name is David Hundley. I am a principal machine learning engineer at State Farm. I enjoy learning about AI and teaching what I learn back to others. I have two daughters. I drive a Tesla Model 3, and my favorite video game series is The Legend of Zelda.


## The Pre-Function Calling Days
Before we demonstrate function calling, let's demonstrate how we used to use prompt engineering and Regex to produce a structure JSON that we can programmatically work with down the road.

In [4]:
# Engineering a prompt to extract as much information from "About Me" as a JSON object
about_me_prompt = f'''
Please extract information as a JSON object. Please look for the following pieces of information.
Name
Job title
Company
Number of children as a single number
Car make
Car model
Favorite video game series

This is the body of text to extract the information from:
{about_me}
'''

In [5]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me_prompt}]
)

In [6]:
# Loading the response as a JSON object
json_response = json.loads(openai_response['choices'][0]['message']['content'])
json_response

{'Name': 'David Hundley',
 'Job title': 'Principal Machine Learning Engineer',
 'Company': 'State Farm',
 'Number of children': 2,
 'Car make': 'Tesla',
 'Car model': 'Model 3',
 'Favorite video game series': 'The Legend of Zelda'}

## Using the New Function Calling Capabilities
Now that we've demonstrated how we used to get structured JSON in the pre-function calling days, let's move into how we can now make use of function calling to extract the same results but in a more consistent manner for our systematic usage. We'll start more simply with a single custom function and then address a few more "advanced" functionalities.

In [21]:
# Defining our initial extract_person_info function
def extract_person_info(name, job_title, num_children):
    '''
    Prints basic "About Me" information

    Inputs:
        name (str): Name of the person
        job_title (str): Job title of the person
        num_chilren (int): The number of children the parent has.
    '''
    
    print(f'This person\'s name is {name}. Their job title is {job_title}, and they have {num_children} children.')

In [8]:
# Defining how we want ChatGPT to call our custom functions
my_custom_functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    }
]

In [10]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me}],
    functions = my_custom_functions,
    function_call = 'auto'
)

print(openai_response)

{
  "id": "chatcmpl-7aSokNbBAHjNFOMAnGqA7uuXsmLjr",
  "object": "chat.completion",
  "created": 1688924938,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extract_person_info",
          "arguments": "{\n  \"name\": \"David Hundley\",\n  \"job_title\": \"Principal Machine Learning Engineer\",\n  \"num_children\": 2\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 147,
    "completion_tokens": 37,
    "total_tokens": 184
  }
}
{'name': 'David Hundley', 'job_title': 'Principal Machine Learning Engineer', 'num_children': 2}


### What if the prompt I submit doesn't contain the information I want to extract per my custom function?
In our original example, our custom function sought to extract three very specific bits of information, and we demonstrated that this worked successfully by passing in my custom "About Me" text as a prompt. But you might be wondering, what happens if you pass in any other prompt that doesn't contain that information?

Recall that we set a parameter in our API client call called function_call that we set to auto. We'll explore this even deeper in the next subsection, but what this parameter is essentially doing is telling ChatGPT to use its best judgment in figuring out when to structure the output for one of our custom functions.

So what happens when we submit a prompt that doesn't match any of our custom functions? Simply put, it defaults to typical behavior as if function calling doesn't exist. Let's test this out with an arbitrary prompt: "How tall is the Eiffel Tower?"

In [11]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': 'How tall is the Eiffel Tower?'}],
    functions = my_custom_functions,
    function_call = 'auto'
)

print(openai_response)

{
  "id": "chatcmpl-7aSywAKIMPCUT2mCNxoJM0OkwYqLJ",
  "object": "chat.completion",
  "created": 1688925570,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The Eiffel Tower is approximately 330 meters (1,083 feet) tall."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 97,
    "completion_tokens": 19,
    "total_tokens": 116
  }
}


In [41]:
# Defining a function to extract only vehicle information
def extract_vehicle_info(vehicle_make, vehicle_model):
    '''
    Prints basic vehicle information

    Inputs:
        - vehicle_make (str): Make of the vehicle
        - vehicle_model (str): Model of the vehicle
    '''
    
    print(f'Vehicle make: {vehicle_make}\nVehicle model: {vehicle_model}')



# Defining a function to extract all information provided in the original "About Me" prompt
def extract_all_info(name, job_title, num_children, vehicle_make, vehicle_model, company_name, favorite_vg_series):
    '''
    Prints the full "About Me" information

    Inputs:
        - name (str): Name of the person
        - job_title (str): Job title of the person
        - num_chilren (int): The number of children the parent has
        - vehicle_make (str): Make of the vehicle
        - vehicle_model (str): Model of the vehicle
        - company_name (str): Name of the company the person works for
        - favorite_vg_series (str): Person's favorite video game series.
    '''
    
    print(f'''
    This person\'s name is {name}. Their job title is {job_title}, and they have {num_children} children.
    They drive a {vehicle_make} {vehicle_model}.
    They work for {company_name}.
    Their favorite video game series is {favorite_vg_series}.
    ''')

In [5]:
# Defining how we want ChatGPT to call our custom functions
my_custom_functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    },
    {
        'name': 'extract_vehicle_info',
        'description': 'Extract the make and model of the person\'s car',
        'parameters': {
            'type': 'object',
            'properties': {
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                }
            }
        }
    },
    {
        'name': 'extract_all_info',
        'description': 'Extract all information about a person including their vehicle make and model',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                },
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                },
                'company_name': {
                    'type': 'string',
                    'description': 'Name of the company the person works for'
                },
                'favorite_vg_series': {
                    'type': 'string',
                    'description': 'Name of the person\'s favorite video game series'
                }
            }
        }
    }
]

Now let's demonstrate what happens when we apply 3 different samples to all of the custom functions.

In [43]:
# Defining a list of samples
samples = [
    str(about_me),
    'My name is David Hundley. I am a principal machine learning engineer, and I have two daughters.',
    'She drives a Kia Sportage.'
]

In [44]:
# Iterating over the three samples
for i, sample in enumerate(samples):
    
    print(f'Sample #{i + 1}\'s results:')

    # Getting the response back from ChatGPT (gpt-3.5-turbo)
    openai_response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = my_custom_functions,
        function_call = 'auto'
    )

    # Printing the sample's response
    print(openai_response)

Sample #1's results:
{
  "id": "chatcmpl-7aTjJLoSCkICSQZM2Eab3HG3IbGca",
  "object": "chat.completion",
  "created": 1688928445,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extract_all_info",
          "arguments": "{\n  \"name\": \"David Hundley\",\n  \"job_title\": \"principal machine learning engineer\",\n  \"num_children\": 2,\n  \"vehicle_make\": \"Tesla\",\n  \"vehicle_model\": \"Model 3\",\n  \"company_name\": \"State Farm\",\n  \"favorite_vg_series\": \"The Legend of Zelda\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 320,
    "completion_tokens": 77,
    "total_tokens": 397
  }
}
Sample #2's results:
{
  "id": "chatcmpl-7aTjLnQMyvRBpfpukfM2SYXKMuI5C",
  "object": "chat.completion",
  "created": 1688928447,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "inde

With each of the respective prompts, ChatGPT selected the correct custom function, and we can specifically note that in the `name` value under `function_call` in the API's response object. In addition to this being a handy way to identify which function to use the arguments for, we can programmatically map our actual custom Python function to this value to run the correct code appropriately.

In [48]:
# Iterating over the three samples
for i, sample in enumerate(samples):
    
    print(f'Sample #{i + 1}\'s results:')

    # Getting the response back from ChatGPT (gpt-3.5-turbo)
    openai_response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = my_custom_functions,
        function_call = 'auto'
    )['choices'][0]['message']

    # Checking to see that a function call was invoked
    if openai_response.get('function_call'):

        # Checking to see which specific function call was invoked
        function_called = openai_response['function_call']['name']

        # Extracting the arguments of the function call
        function_args = json.loads(openai_response['function_call']['arguments'])

        # Invoking the proper functions
        if function_called == 'extract_person_info':
            extract_person_info(*list(function_args.values()))
        elif function_called == 'extract_vehicle_info':
            extract_vehicle_info(*list(function_args.values()))
        elif function_called == 'extract_all_info':
            extract_all_info(*list(function_args.values()))

Sample #1's results:

    This person's name is David Hundley. Their job title is principal machine learning engineer, and they have 2 children.
    They drive a Tesla Model 3.
    They work for State Farm.
    Their favorite video game series is The Legend of Zelda.
    
Sample #2's results:
This person's name is David Hundley. Their job title is Principal Machine Learning Engineer, and they have 2 children.
Sample #3's results:
Vehicle make: Kia
Vehicle model: Sportage


## OpenAI Function Calling with LangChain
Given its popularity amongst the Generative AI community, I thought I'd re-visit this notebook and add some code to show how you might make use of this exact same functionality in LangChain

In [6]:
# Importing the LangChain objects
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains.openai_functions import create_structured_output_chain

In [17]:
# Setting the proper instance of the OpenAI model
llm = ChatOpenAI(model = 'gpt-3.5-turbo-0613')

# Setting a LangChain ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_template('{my_prompt}')

# Setting the JSON schema for extracting vehicle information
langchain_json_schema = {
    'name': 'extract_vehicle_info',
    'description': 'Extract the make and model of the person\'s car',
    'type': 'object',
    'properties': {
        'vehicle_make': {
            'title': 'Vehicle Make',
            'type': 'string',
            'description': 'Make of the person\'s vehicle'
        },
        'vehicle_model': {
            'title': 'Vehicle Model',
            'type': 'string',
            'description': 'Model of the person\'s vehicle'
        }
    }
}

In [18]:
# Defining the LangChain chain object for function calling
chain = create_structured_output_chain(output_schema = langchain_json_schema,
                                       llm = llm,
                                       prompt = chat_prompt_template)

In [19]:
# Getting results with a demo prompt
print(chain.run(my_prompt = 'I drive a Tesla Model 3'))

{'vehicle_make': 'Tesla', 'vehicle_model': 'Model 3'}
